In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256
import codecs


from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

1.1080374717712402

In [3]:
# casos = casos_confirmados.fix_mun_resid_casos('mun_atend')
# obitos = casos_confirmados.fix_mun_resid_obitos()
casos = casos_confirmados.get_casos()
casos = casos.drop(columns=['obito','data_obito','excluir'])
obitos = casos_confirmados.get_obitos()

print(f"{len(casos)};{len(obitos)}")
obitos.loc[~obitos['hash'].isin(casos['hash'])]

486843;8916


,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,hash,hash_less,hash_more,hash_dt_obito
35,36,4118501,ADAO DA SILVA,M,70,PATO BRANCO,07,2020-04-13,ADAODASILVA70PATOBRANCO,ADAODASILVA69PATOBRANCO,ADAODASILVA71PATOBRANCO,ADAODASILVA20200413000000
121,122,4125506,IVO DA SILVA,M,57,SAO JOSE DOS PINHAIS,02,2020-05-10,IVODASILVA57SAOJOSEDOSPINHAIS,IVODASILVA56SAOJOSEDOSPINHAIS,IVODASILVA58SAOJOSEDOSPINHAIS,IVODASILVA20200510000000
138,139,4125506,APARECIDA GOMES DE MORAIS,F,76,SAO JOSE DOS PINHAIS,02,2020-05-17,APARECIDAGOMESDEMORAIS76SAOJOSEDOSPINHAIS,APARECIDAGOMESDEMORAIS75SAOJOSEDOSPINHAIS,APARECIDAGOMESDEMORAIS77SAOJOSEDOSPINHAIS,APARECIDAGOMESDEMORAIS20200517000000
143,144,4126678,VANILDA GONCALVES DOS SANTOS,F,38,TAMARANA,17,2020-05-17,VANILDAGONCALVESDOSSANTOS38TAMARANA,VANILDAGONCALVESDOSSANTOS37TAMARANA,VANILDAGONCALVESDOSSANTOS39TAMARANA,VANILDAGONCALVESDOSSANTOS20200517000000
144,145,4118402,JOSE PAULO DIAS DA SILVA,M,53,PARANAVAI,14,2020-05-18,JOSEPAULODIASDASILVA53PARANAVAI,JOSEPAULODIASDASILVA52PARANAVAI,JOSEPAULODIASDASILVA54PARANAVAI,JOSEPAULODIASDASILVA20200518000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5233,5244,4119905,CELSO CERQUEIRA LIMA,M,56,PONTA GROSSA,03,2020-10-15,CELSOCERQUEIRALIMA56PONTAGROSSA,CELSOCERQUEIRALIMA55PONTAGROSSA,CELSOCERQUEIRALIMA57PONTAGROSSA,CELSOCERQUEIRALIMA20201015000000
5273,5284,4101804,ADEMIR LEMOS,M,67,ARAUCARIA,02,2020-10-17,ADEMIRLEMOS67ARAUCARIA,ADEMIRLEMOS66ARAUCARIA,ADEMIRLEMOS68ARAUCARIA,ADEMIRLEMOS20201017000000
5313,5324,4104204,ANTONIO KICHILEVCZ,M,86,CAMPO LARGO,02,2020-10-19,ANTONIOKICHILEVCZ86CAMPOLARGO,ANTONIOKICHILEVCZ85CAMPOLARGO,ANTONIOKICHILEVCZ87CAMPOLARGO,ANTONIOKICHILEVCZ20201019000000
5342,5353,4119152,NILSON SUTIL SANTANA,M,49,PINHAIS,02,2020-10-20,NILSONSUTILSANTANA49PINHAIS,NILSONSUTILSANTANA48PINHAIS,NILSONSUTILSANTANA50PINHAIS,NILSONSUTILSANTANA20201020000000


In [4]:
obitos.loc[obitos['hash'].isin(casos['hash_less']),'idade'] += 1
obitos.loc[obitos['hash'].isin(casos['hash_less']),'hash'] = obitos.loc[obitos['hash'].isin(casos['hash_less'])].apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['municipio']), axis=1)

In [5]:
obitos.loc[obitos['hash'].isin(casos['hash_more']),'idade'] -= 1
obitos.loc[obitos['hash'].isin(casos['hash_more']),'hash'] = obitos.loc[obitos['hash'].isin(casos['hash_more'])].apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['municipio']), axis=1)

In [6]:
obitos.loc[~obitos['hash'].isin(casos['hash'])]

,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,hash,hash_less,hash_more,hash_dt_obito


In [7]:
obitos = obitos.rename(columns={'data_do_obito': 'data_obito'})
obitos['obito'] = 'SIM'

casos = pd.merge(left=casos, right=obitos[['obito','data_obito','hash']], on='hash', how='left')

casos.loc[casos['obito'].isnull(), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'data_obito'] = pd.NaT

print(f"{len(casos)};{len(casos.loc[casos['obito']=='SIM'])}")

486843;8916


In [8]:
def get_better(cc):
    scores = np.zeros(len(cc))

    for i, (_, cc_row) in enumerate(cc.iterrows()):
        # if cc_row['obito'] == 'SIM':
        scores[i] = i

    i = np.argmax(scores)
    return cc.iloc[i].name

In [9]:
casos['nome_hash'] = casos['nome'].apply(normalize_hash)
new_casos = casos.sort_values('nome_hash')[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','nome_hash','hash_dt_diag']]

obitos['nome_hash'] = obitos['nome'].apply(normalize_hash)
new_obitos = obitos.sort_values('nome_hash')[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito','nome_hash','hash_dt_obito']]

In [10]:
casos_duplicados = new_casos.loc[new_casos.duplicated('hash_dt_diag',keep=False)].sort_values('nome_hash')

writer = pd.ExcelWriter(join(output,'casos_duplicados.xlsx'),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos_duplicados.to_excel(writer,sheet_name="casos_duplicados",index=None)
worksheet = writer.sheets["casos_duplicados"]
auto_fit_columns(worksheet,casos_duplicados)

writer.save()
writer.close()

In [11]:
casos['excluir'] = 'NAO'
manter = []
casos_duplicados = casos_duplicados.sort_values('ordem')

for cc_group in casos_duplicados.groupby('hash_dt_diag'):
    
    idx = get_better(cc_group[1])
    manter.append(idx)

duplicados = set(casos_duplicados.index.tolist())
duplicados = duplicados - set(manter)

casos.loc[duplicados,'excluir'] = 'SIM'

In [12]:
obitos_duplicados = new_obitos.loc[new_obitos.duplicated('hash_dt_obito',keep=False)].sort_values('nome_hash')

writer = pd.ExcelWriter(join(output,'obitos_duplicados.xlsx'),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

obitos_duplicados.to_excel(writer,sheet_name="obitos_duplicados",index=None)
worksheet = writer.sheets["obitos_duplicados"]
auto_fit_columns(worksheet,obitos_duplicados)

writer.save()
writer.close()

In [13]:
obitos['excluir'] = 'NAO'
manter = []
obitos_duplicados = obitos_duplicados.sort_values('ordem')

for cc_group in obitos_duplicados.groupby('hash_dt_obito'):
    
    idx = get_better(cc_group[1])
    manter.append(idx)

duplicados = set(obitos_duplicados.index.tolist())
duplicados = duplicados - set(manter)

obitos.loc[duplicados,'excluir'] = 'SIM'

In [14]:
new_casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir']]
new_obitos = obitos[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito','excluir']]

In [15]:
writer = pd.ExcelWriter(join(output,"Casos confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()

In [16]:
writer = pd.ExcelWriter(join(output,f"Casos Confirmados Regionais.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

for grupo_casos, grupo_obitos in zip(new_casos.groupby('rs'),new_obitos.groupby('rs')):
    grupo_casos[1].to_excel(writer,sheet_name=f"casos_rs{grupo_casos[0]}",index=None)
    grupo_obitos[1].to_excel(writer,sheet_name=f"obitos_rs{grupo_casos[0]}",index=None)
    worksheet = writer.sheets[f"casos_rs{grupo_casos[0]}"]
    auto_fit_columns(worksheet,grupo_casos[1])
    worksheet = writer.sheets[f"obitos_rs{grupo_casos[0]}"]
    auto_fit_columns(worksheet,grupo_obitos[1])

new_casos.loc[new_casos['rs'].isnull()].to_excel(writer,sheet_name='casos_fora',index=None)
new_obitos.loc[new_obitos['rs'].isnull()].to_excel(writer,sheet_name='obitos_fora',index=None)

writer.save()
writer.close()